In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.ensemble import VotingClassifier

# Importing Data

In [2]:
data = pickle.load(open('data_face_features_emotion.pickle', mode='rb'))

In [3]:
X = np.array(data['data'])
y = np.array(data['label'])

In [4]:
X.shape, y.shape

((8184, 1, 128), (8184,))

In [5]:
X = X.reshape(-1,128)
X.shape

(8184, 128)

# Splitting Data Into Training and Testing Set

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [8]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((6547, 128), (1637, 128), (6547,), (1637,))

In [9]:
print(X_test)

[[ 1.7084204e-01  6.2720649e-02 -3.1688280e-02 ... -6.6123895e-02
   3.9222505e-02 -2.9056912e-02]
 [ 5.0236665e-02  1.2073725e-01 -1.4319929e-01 ...  5.7972234e-02
   7.8380108e-05 -4.4495326e-02]
 [ 9.0904668e-02  5.9862979e-02  4.0252090e-02 ...  8.7137476e-02
   2.1865651e-01 -1.3280847e-02]
 ...
 [ 9.6387222e-02  7.8339070e-02 -1.1227283e-02 ...  8.9317277e-02
   1.2892395e-02 -5.6562062e-02]
 [ 5.8613464e-02  7.3899239e-02 -1.3305034e-01 ... -1.7231515e-02
   8.0799893e-02  4.2741671e-02]
 [ 3.5729777e-02 -5.8938131e-02  4.6944343e-02 ...  9.1232970e-02
   1.7257439e-01  1.4514689e-01]]


In [10]:
print(y_test)

['angry' 'sad' 'angry' ... 'happy' 'surprise' 'sad']


# Training Model

## Logistic Regression

In [11]:
logistic_classifier = LogisticRegression()
logistic_classifier.fit(X_train, y_train)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
def get_report(model, X_train, y_train, X_test, y_test):
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)

    # F1 Score
    f1_score_train = f1_score(y_train, y_pred_train,average='macro')
    f1_score_test = f1_score(y_test,y_pred_test,average='macro')

    print('Accuracy Train =', accuracy_train)
    print('Accuracy Test =', accuracy_test)
    print('F1 Score Train =', f1_score_train)
    print('F1 Score Test =', f1_score_test)

In [13]:
get_report(logistic_classifier, X_train, y_train, X_test, y_test)

Accuracy Train = 0.33954482969298916
Accuracy Test = 0.31337813072693954
F1 Score Train = 0.31324730458650085
F1 Score Test = 0.28988498705302196


## Support Vector Machine

In [14]:
model_svc = SVC(probability=True)
model_svc.fit(X_train, y_train)

SVC(probability=True)

In [15]:
get_report(model_svc, X_train, y_train, X_test, y_test)

Accuracy Train = 0.4935084771651138
Accuracy Test = 0.3860720830788027
F1 Score Train = 0.48611325137893946
F1 Score Test = 0.3782059269537095


## Random Forest

In [16]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)

RandomForestClassifier()

In [17]:
get_report(model_rf, X_train, y_train, X_test, y_test)

Accuracy Train = 0.9993890331449519
Accuracy Test = 0.5540623091020159
F1 Score Train = 0.9993955328746674
F1 Score Test = 0.5554523294330591


# Voting Classifier

In [18]:
model_voting = VotingClassifier(estimators = [
    ('logisitc', LogisticRegression()),
    ('svm',SVC(probability=True)),
    ('rf',RandomForestClassifier())
],voting='soft',weights=[2,3,1])

In [19]:
model_voting.fit(X_train,y_train)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('logisitc', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [20]:
get_report(model_voting, X_train, y_train, X_test, y_test)

Accuracy Train = 0.7452268214449366
Accuracy Test = 0.4520464263897373
F1 Score Train = 0.7425405242749988
F1 Score Test = 0.43885407361175616


# Parameter Tuning

In [21]:
from sklearn.model_selection import GridSearchCV

In [22]:
model_grid = GridSearchCV(model_voting,param_grid={
    'svm__C':[3,5,7,10],
    'svm__gamma':[0.1,0.3,0.5],
    'rf__n_estimators':[5,10,20],
    "rf__max_depth":[3,5,7],
    'voting':['soft','hard']
},scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [23]:
model_grid.fit(X_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  11.8s


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  11.3s


...

[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=hard; total time=  11.9s


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=hard; total time=  11.9s


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logisitc',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [25]:
model_best_estimator = model_grid.best_estimator_

In [26]:
model_grid.best_score_

0.38643475770819674

# Saving Model

In [27]:
pickle.dump(model_best_estimator,open('./models/machinelearning_face_emotion.pkl',mode='wb'))